In [1]:
import sys, IPython

install_needed = True
#install_needed = False

if install_needed:
    print("===> Installing deps and restarting kernel. Please change 'install_needed = False' and run this code cell again.")
    !{sys.executable} -m pip install -U sagemaker locust pyngrok torch==1.11 transformers==4.17
    #IPython.Application.instance().kernel.do_shutdown(True)
else:
    import sagemaker, transformers, torch
    print(f'SageMaker API version={sagemaker.__version__}')

===> Installing deps and restarting kernel. Please change 'install_needed = False' and run this code cell again.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 1.6 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.6/750.6 MB 1.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 42.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 22.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 27.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.8/128.8 kB 3.1 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 91.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 16.3 MB/s eta 0:00:0000:01
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.219.0
    Uninstalling sagemaker-2.219

In [2]:
!nvidia-smi

Sun May 19 08:07:30 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.161.08             Driver Version: 535.161.08   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       On  | 00000000:00:1E.0 Off |                    0 |
| N/A   24C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
import boto3
import sagemaker

# SageMaker 세션과 기본 S3 버킷 초기화
sess = sagemaker.Session()
bucket = sess.default_bucket()

# Boto3를 통한 SageMaker 클라이언트 설정
sm_client = boto3.client("sagemaker")

# 실행 역할 및 리전 설정
role = sagemaker.get_execution_role()
boto_session = boto3.session.Session()
region = boto_session.region_name

# 확인을 위한 출력
print((bucket, role, region))

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
('sagemaker-ap-northeast-2-471112711710', 'arn:aws:iam::471112711710:role/service-role/AmazonSageMaker-ExecutionRole-20240512T210165', 'ap-northeast-2')


In [4]:
import os
import shutil
import tarfile
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# 모델과 토크나이저 다운로드
model_name = 'gpt2'
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# 로컬 디렉터리에 저장
model_dir = './model'
os.makedirs(model_dir, exist_ok=True)
model.save_pretrained(model_dir)
tokenizer.save_pretrained(model_dir)

# 현재 위치의 requirements.txt 파일을 model_dir에 복사
requirements_path = './requirements.txt'
if os.path.exists(requirements_path):
    shutil.copy(requirements_path, os.path.join(model_dir, 'requirements.txt'))
else:
    print("requirements.txt 파일을 찾을 수 없습니다.")

# tar.gz 파일로 압축
with tarfile.open('model.tar.gz', 'w:gz') as tar:
    for file_name in os.listdir(model_dir):
        tar.add(os.path.join(model_dir, file_name), arcname=file_name)

# S3에 업로드
import boto3
s3 = boto3.client('s3')
bucket_name = 'dawonawss3'
s3.upload_file('model.tar.gz', bucket_name, 'model.tar.gz')


In [6]:
model = GPT2LMHeadModel.from_pretrained(model_dir)

In [4]:
bucket_name = 'dawonawss3'

In [7]:
from sagemaker.huggingface import HuggingFaceModel

# HuggingFaceModel 객체 생성
huggingface_model = HuggingFaceModel(
    model_data='s3://{}/model.tar.gz'.format(bucket_name),  # S3에 저장된 모델 경로
    role=role,  # SageMaker 역할
    entry_point='inference.py',  # 모델 및 추론 스크립트 파일
    transformers_version='4.17.0',  # 트랜스포머 라이브러리 버전
    pytorch_version='1.10.2',  # 파이토치 버전
    py_version='py38',  # 파이썬 버전
    #image_uri='471112711710.dkr.ecr.ap-northeast-2.amazonaws.com/my_pytorch_repository:latest',
    #name=bucket_name,  # 모델 이름 명시적으로 지정
)

# 엔드포인트 배포
predictor = huggingface_model.deploy(initial_instance_count=1, instance_type='ml.g4dn.xlarge')

-------!

In [6]:
predictor.delete_endpoint()